In [31]:
import numpy as np
import bitarray

In [32]:
fileObject = open("input.txt", "r")
data = fileObject.read()
data

"Ceci est un test. Je m'appelle Antoine et j'écris des accents pour faire des tests."

In [52]:
def string_to_nparray_of_bits(s):
    ba = bitarray.bitarray()
    ba.frombytes(data.encode('utf-8'))
    nparray = np.array(list(ba))
    return np.where(nparray == 0, -1, nparray)

In [53]:
tmp = string_to_nparray_of_bits(data)
tmp

array([-1,  1, -1, -1, -1, -1,  1,  1, -1,  1,  1, -1, -1,  1, -1,  1, -1,
        1,  1, -1, -1, -1,  1,  1, -1,  1,  1, -1,  1, -1, -1,  1, -1, -1,
        1, -1, -1, -1, -1, -1, -1,  1,  1, -1, -1,  1, -1,  1, -1,  1,  1,
        1, -1, -1,  1,  1, -1,  1,  1,  1, -1,  1, -1, -1, -1, -1,  1, -1,
       -1, -1, -1, -1, -1,  1,  1,  1, -1,  1, -1,  1, -1,  1,  1, -1,  1,
        1,  1, -1, -1, -1,  1, -1, -1, -1, -1, -1, -1,  1,  1,  1, -1,  1,
       -1, -1, -1,  1,  1, -1, -1,  1, -1,  1, -1,  1,  1,  1, -1, -1,  1,
        1, -1,  1,  1,  1, -1,  1, -1, -1, -1, -1,  1, -1,  1,  1,  1, -1,
       -1, -1,  1, -1, -1, -1, -1, -1, -1,  1, -1, -1,  1, -1,  1, -1, -1,
        1,  1, -1, -1,  1, -1,  1, -1, -1,  1, -1, -1, -1, -1, -1, -1,  1,
        1, -1,  1,  1, -1,  1, -1, -1,  1, -1, -1,  1,  1,  1, -1,  1,  1,
       -1, -1, -1, -1,  1, -1,  1,  1,  1, -1, -1, -1, -1, -1,  1,  1,  1,
       -1, -1, -1, -1, -1,  1,  1, -1, -1,  1, -1,  1, -1,  1,  1, -1,  1,
        1, -1, -1, -1,  1

In [54]:
def bits_to_string(bs):
    return bitarray.bitarray(list(np.where(bs == -1, 0, bs))).tobytes().decode('utf-8')

In [56]:
bits_to_string(tmp)

"Ceci est un test. Je m'appelle Antoine et j'écris des accents pour faire des tests."

In [57]:
def channel(chanInput):
    chanInput = np.clip(chanInput,-1,1)
    erasedIndex = np.random.randint(3)
    chanInput[erasedIndex:len(chanInput):3] = 0
    return chanInput + np.sqrt(10)*np.random.randn(len(chanInput))

In [58]:
def channel_no_noise(chanInput):
    chanInput = np.clip(chanInput,-1,1)
    erasedIndex = np.random.randint(3)
    chanInput[erasedIndex:len(chanInput):3] = 0
    return chanInput

In [59]:
tmp = string_to_nparray_of_bits(data)
tmp = channel_no_noise(tmp)
tmp

array([-1,  0, -1, -1,  0, -1,  1,  0, -1,  1,  0, -1, -1,  0, -1,  1,  0,
        1,  1,  0, -1, -1,  0,  1, -1,  0,  1, -1,  0, -1, -1,  0, -1, -1,
        0, -1, -1,  0, -1, -1,  0,  1,  1,  0, -1,  1,  0,  1, -1,  0,  1,
        1,  0, -1,  1,  0, -1,  1,  0,  1, -1,  0, -1, -1,  0, -1,  1,  0,
       -1, -1,  0, -1, -1,  0,  1,  1,  0,  1, -1,  0, -1,  1,  0, -1,  1,
        0,  1, -1,  0, -1,  1,  0, -1, -1,  0, -1, -1,  0,  1,  1,  0,  1,
       -1,  0, -1,  1,  0, -1, -1,  0, -1,  1,  0,  1,  1,  0, -1, -1,  0,
        1, -1,  0,  1,  1,  0,  1, -1,  0, -1, -1,  0, -1,  1,  0,  1, -1,
        0, -1,  1,  0, -1, -1,  0, -1, -1,  0, -1, -1,  0, -1,  1,  0, -1,
        1,  0, -1, -1,  0, -1,  1,  0, -1,  1,  0, -1, -1,  0, -1, -1,  0,
        1, -1,  0,  1, -1,  0, -1, -1,  0, -1, -1,  0,  1,  1,  0,  1,  1,
        0, -1, -1,  0,  1, -1,  0,  1,  1,  0, -1, -1,  0, -1,  1,  0,  1,
       -1,  0, -1, -1,  0,  1,  1,  0, -1,  1,  0,  1, -1,  0,  1, -1,  0,
        1, -1,  0, -1,  1

In [60]:
bits_to_string(tmp)

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xa0 in position 45: invalid start byte